In [1]:
!pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 13.7 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/babthamotharan/rpunct.git@patch-2

  Cloning https://github.com/babthamotharan/rpunct.git (to revision patch-2) to /tmp/pip-req-build-r9qt1gzp
  Running command git clone --filter=blob:none --quiet https://github.com/babthamotharan/rpunct.git /tmp/pip-req-build-r9qt1gzp
  Running command git checkout -b patch-2 --track origin/patch-2
  Switched to a new branch 'patch-2'
  Branch 'patch-2' set up to track remote branch 'patch-2' from 'origin'.
  Resolved https://github.com/babthamotharan/rpunct.git to commit a87b93410ca782657abb4e34df9159e6e47ac9ec
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install openai gradio

In [4]:
import gradio as gr
import os
import re
from youtube_transcript_api import YouTubeTranscriptApi
from rpunct import RestorePuncts
import openai
from openai import OpenAI

In [5]:
# Initialize the punctuation restoration model
rpunct = RestorePuncts()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/530 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
# Function to extract video ID from YouTube URL
def get_video_id(url_link):
    # Handle different YouTube URL formats
    if "youtu.be" in url_link:
        return url_link.split("/")[-1].split("?")[0]
    elif "youtube.com/watch" in url_link:
        match = re.search(r"v=([^&]+)", url_link)
        return match.group(1) if match else None
    else:
        return None

In [7]:
# Function to get transcript from YouTube video
def get_transcript(video_url):
    try:
        video_id = get_video_id(video_url)
        if not video_id:
            return "Invalid YouTube URL. Please provide a valid YouTube video link."

        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = " ".join([line['text'] for line in transcript])

        # Restore punctuation for better readability
        restored_text = rpunct.punctuate(transcript_text)
        return restored_text
    except Exception as e:
        return f"Error fetching transcript: {str(e)}"

In [8]:
# Function to summarize transcript using OpenAI
def summarize_transcript(transcript, api_key):
    if not api_key:
        return "Please provide an OpenAI API key."

    try:
        client = OpenAI(api_key=api_key)

        # Create a prompt for summarization
        prompt = f"Summarize this YouTube video transcript concisely:\n\n{transcript[:8000]}"  # Limit text to avoid token limit

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes YouTube video content."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )

        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating summary: {str(e)}"

In [9]:
# Function to get answer from chatbot
def get_chatbot_response(message, transcript_text, api_key):
    if not transcript_text:
        return "Please provide a YouTube video link first to load the content."

    if not api_key:
        return "Please provide an OpenAI API key."

    try:
        client = OpenAI(api_key=api_key)

        # Create context-aware prompt
        prompt = f"""
        Based on the following YouTube video transcript, answer the user's question.

        Transcript: {transcript_text[:8000]}

        User's Question: {message}

        Please provide a specific answer based only on the information in the transcript. If the answer isn't in the transcript, say so.
        """

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions about YouTube video content."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )

        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating response: {str(e)}"

In [10]:
css = """
.gradio-container {
    background-color: #ffffff; /* Already white */
    font-family: 'Roboto', sans-serif;
    display: flex;
    justify-content: center;
    padding: 20px;
}

.gradio-interface {
    max-width: 900px;
    margin: auto;
    padding: 25px;
    background: rgba(0, 0, 0, 0.85);
    border-radius: 15px;
    animation: fadeIn 0.5s ease-in-out;
}

.main-header {
    background-color: #E6E6FA; /* Keeping this as is */
    color: #000000; /* Changed text color to black for better contrast with light background */
    padding: 15px 20px;
    border-radius: 12px;
    margin-bottom: 20px;
    box-shadow: 0 6px 12px rgba(0, 0, 0, 0.15);
    text-align: center;
}

.main-header img {
    height: 40px;
    vertical-align: middle;
    margin-right: 10px;
}

.youtube-button {
    background-color: #FF6600 !important; /* Orange, no change needed */
    color: white !important;
    font-weight: 600;
    border-radius: 8px;
    padding: 10px 20px;
    transition: all 0.3s ease-in-out;
    box-shadow: 0 4px 6px rgba(150, 0, 0, 0.3); /* Adjusted shadow color, no change needed */
}

.youtube-button:hover {
    background-color: #FF6600 !important; /* Orange, no change needed */
    transform: scale(1.02);
}

.page-content {
    padding: 20px;
    border-radius: 12px;
    background-color: #ffffff; /* Already white */
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.05); /* Very light shadow, no change needed */
    animation: fadeIn 0.3s ease-in-out;
}

.video-title {
    font-size: 22px;
    font-weight: bold;
    margin-bottom: 10px;
    color: #000000; /* Changed text color to black */
}

.info-text {
    font-size: 16px;
    color: #000000; /* Changed text color to black */
    margin-bottom: 20px;
}

@keyframes fadeIn {
    from { opacity: 0; transform: translateY(10px); }
    to { opacity: 1; transform: translateY(0); }
}
"""

In [11]:
# Define Gradio app with multiple pages
with gr.Blocks(css=css) as app:
    # Shared state across pages
    transcript_state = gr.State("")
    openai_api_key = gr.State("")
    video_url_state = gr.State("")

    with gr.Tabs() as tabs:
        # Welcome Page
        with gr.TabItem("Welcome"):
            gr.HTML("""
    <div class="main-header">
        <img src="https://upload.wikimedia.org/wikipedia/commons/b/b8/YouTube_Logo_2017.svg" alt="YouTube" />
        <h1 style="display:inline-block; vertical-align: middle;">Video Summarizer & Chatbot</h1>
    </div>
""")


            with gr.Group(elem_classes="page-content"):
                gr.Markdown("""
                    ## Welcome to the YouTube Video Summarizer!

                    This tool helps you extract key information from YouTube videos without watching the entire content.

                    ### Features:
                    - 📝 Get automatic summaries of any YouTube video with subtitles
                    - 💬 Ask questions about the video content
                    - 🚀 Save time by quickly understanding the main points

                    ### How to Use:
                    1. Go to the "Summarize" tab
                    2. Enter your OpenAI API key (it's secured and only used for processing)
                    3. Paste a YouTube video URL
                    4. Get a concise summary
                    5. Go to the "Chat" tab to ask specific questions about the video

                    Let's get started!
                """)

                with gr.Row():
                    gr.Button("Start Summarizing!", elem_classes="youtube-button").click(
                        lambda: tabs.set(1), # Use tabs.set(index) to switch tabs
                        inputs=None,
                        outputs=tabs
                    )

        # Summarize Page
        with gr.TabItem("Summarize"):
            gr.HTML("""
                <div class="main-header">
                    <h1>📋 Summarize YouTube Video</h1>
                </div>
            """)

            with gr.Group(elem_classes="page-content"):
                api_key_input = gr.Textbox(
                    label="OpenAI API Key (required)",
                    placeholder="Enter your OpenAI API key here...",
                    type="password"
                )

                video_url = gr.Textbox(
                    label="YouTube Video URL",
                    placeholder="Paste the YouTube video URL here...",
                    lines=1
                )

                with gr.Row():
                    summarize_btn = gr.Button("Summarize Video", elem_classes="youtube-button")

                summary_output = gr.Textbox(
                    label="Video Summary",
                    placeholder="Summary will appear here...",
                    lines=10,
                    interactive=False
                )

                gr.Markdown("### Want to learn more? Chat with the video content!")

                chat_btn = gr.Button("Go to Chat", elem_classes="youtube-button")

        # Chat Page
        with gr.TabItem("Chat"):
            gr.HTML("""
                <div class="main-header">
                    <h1>💬 Chat with Video Content</h1>
                </div>
            """)

            with gr.Group(elem_classes="page-content"):
                gr.Markdown("Ask questions about the YouTube video content!")

                video_info = gr.Textbox(label="Current Video", interactive=False)

                chatbot = gr.Chatbot(
                    label="Chat with Video Content",
                    height=400
                )

                msg = gr.Textbox(
                    label="Your Question",
                    placeholder="Ask something about the video...",
                    lines=2
                )

                with gr.Row():
                    submit_btn = gr.Button("Ask", elem_classes="youtube-button")
                    clear_btn = gr.Button("Clear Chat")

    # Function to handle summarization process
    def summarize_video(url, api_key):
        if not url:
            return "Please enter a YouTube video URL.", "", "", ""

        # Store API key in state
        api_key_state = api_key

        # Get transcript
        transcript = get_transcript(url)
        if transcript.startswith("Error"):
            return transcript, "", url, api_key_state

        # Generate summary
        summary = summarize_transcript(transcript, api_key)

        return summary, transcript, url, api_key_state

    # Function to handle chat interactions
    def respond(message, chat_history, transcript, api_key, video_url):
        if not transcript:
            response = "Please summarize a YouTube video first."
            chat_history.append((message, response))
            return "", chat_history

        # Get response from the model
        response = get_chatbot_response(message, transcript, api_key)

        # Update chat history
        chat_history.append((message, response))
        return "", chat_history

    # Set up event handlers
    summarize_btn.click(
        fn=summarize_video,
        inputs=[video_url, api_key_input],
        outputs=[summary_output, transcript_state, video_url_state, openai_api_key]
    )

    chat_btn.click(
        lambda: gr.Tabs(selected=2),
        inputs=None,
        outputs=tabs
    )

    submit_btn.click(
        fn=respond,
        inputs=[msg, chatbot, transcript_state, openai_api_key, video_url_state],
        outputs=[msg, chatbot]
    )

    clear_btn.click(lambda: [], None, chatbot)

    # Update video info when entering chat tab
    def update_video_info(url):
        if url:
            return f"Currently chatting about: {url}"
        else:
            return "No video selected. Please go to the Summarize tab first."

    # Connect video URL state to video info display
    video_url_state.change(
        fn=update_video_info,
        inputs=[video_url_state],
        outputs=[video_info]
    )


/tmp/ipython-input-3088416289.py:95: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


In [12]:
# Launch the app
if __name__ == "__main__":
    app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a17e27ad5c939fd1b1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
